In [10]:
from pyspark.sql import SparkSession
import dotenv
import os

env_path = dotenv.find_dotenv()
dotenv.load_dotenv(env_path)

spark = SparkSession.builder \
        .master("local") \
        .appName("hadoop-ncloud") \
        .config("spark.driver.extraClassPath", "./jdbc/mysql-connector-j-8.4.0.jar:./jdbc/postgresql-42.7.3.jar") \
        .getOrCreate()

sc = spark.sparkContext
access_key = os.environ.get("ACCESS_KEY_ID")
secret_key = os.environ.get("ACCESS_SECRET_KEY")

sc._jsc.hadoopConfiguration().set("fs.s3a.access.key", access_key)
sc._jsc.hadoopConfiguration().set("fs.s3a.secret.key", secret_key)
sc._jsc.hadoopConfiguration().set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
sc._jsc.hadoopConfiguration().set("fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
sc._jsc.hadoopConfiguration().set("fs.s3a.connection.ssl.enabled", "false")
# sc._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "http://kr.objectstorage.ncloud.com")

In [2]:
df = spark.read.parquet("s3a://personal-golight-image-bucket/postgresql/fake_dataframe/")

In [3]:
def write_mysql(df, db_name, table_name):
    df.write \
    .mode("append") \
    .format("jdbc") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("url", f"jdbc:mysql://dockercompose-mysql-1:3306/{db_name}") \
    .option("dbtable", table_name) \
    .option("user", "hyunsoo") \
    .option("password", "910506") \
    .save()

In [4]:
write_mysql(df, "dataops", "s3data")

In [4]:
pdf = df.groupby("city").count()

pdf.coalesce(1).write.mode('overwrite').parquet("s3a://personal-golight-image-bucket/gazuaaaaaaa")

In [6]:
df.createOrReplaceTempView("fake")

In [7]:
def qq(query):
    return spark.sql(query)

In [8]:
_query = """
SELECT *,
CASE 
    WHEN age <40 THEN '젊은이'
    WHEN age >=40 AND age < 70 THEN '중년'
    ELSE '노년'
END AS analy
FROM fake
"""

qq(_query).show()

+--------------------+------+--------------------------------+---+-----+--------------+----------+---+------------+----------+----+---+---+------+
|                uuid|  name|                             job|sex|blood|          city|birth_year|age|age_category|batch_date|yyyy| mm| dd| analy|
+--------------------+------+--------------------------------+---+-----+--------------+----------+---+------------+----------+----+---+---+------+
|PscFahxA7ZipcqJiy...|김순옥|                          수금원|  F|    A|        경기도|      1936| 88|        80대|  20240114|2024|  1| 14|  노년|
|aFcBovFwKKGYuZ2Np...|박경자|전기 부품 및 제품제조 기계조작원|  F|   AB|    부산광역시|      2004| 20|        20대|  20240114|2024|  1| 14|젊은이|
|HhxVzky52G3iLd8vS...|김도윤|          보조 교사 및 기타 교사|  M|    A|    울산광역시|      1959| 65|        60대|  20240114|2024|  1| 14|  중년|
|daTdeSWJnf743fZpN...|이성호|    국가/지방 및 공공행정 사무원|  M|   AB|제주특별자치도|      1995| 29|        20대|  20240114|2024|  1| 14|젊은이|
|UtxCAwqnYXjBoRcJJ...|박채원|             섀시 조립 및 설치원|  F|